Bis jetzt haben wir die Gradienten der Kostenfunktion noch selbst hergeleitet und anschliessend implementiert um die Optimierung durchzuführen. Diese Schritte lassen sich zum Glück auch sehr einfach automatisieren, zum Beispiel mit Pytorch.

Wir haben unten das Logistic-Regression Beispiel der vorangegangenen Aufgaben in Pytorch implementiert. Das Modell, d.h. die lineare Klassifikationsfunktion, wird durch `nn.Linear` realisiert. Die Kostenfunktion ist in `nn.BCEWithLogitsLoss` implementiert. Noch besser: \
die Ableitung wird automatisch von Pytorch mit der Funktion `backward` berechnet. Ihr braucht den Gradienten also nicht mehr per Hand berechnen. Ein Optimierungsschritt, d.h. die Anpassung der Modellparameter in Richtung des (negativen) Gradienten, wird mit der Funktion\
 `step` durchgeführt.

Neben dem normalen Gradientabstieg können wir nun auch verwandte Abstiegsverfahren nutzen. Diese Verfahren heißen zum Beispiel `Adam` oder `RMSprop` und modifizieren den Gradientabstieg so, dass das Minimum der Kostenfunktion teilweise deutlich schneller und zuverlässig\
er gefunden werden kann.

Aufgabe: Finde passende Werte für die sogenannten Hyperparameter `n_steps` und `learning_rate`.

Hier findest du Video-Tutorials zu Pytorch:
[Das erste eigene Neuronale Netz](https://www.youtube.com/watch?v=GBzojftwfGQ)
[Das Netz füttern](https://www.youtube.com/watch?v=u6V69py5Aps)
[Das Netz lernen lassen](https://www.youtube.com/watch?v=F4V2617urQs)


In [ ]:
import torch
import numpy as np
import torch.nn as nn

D = np.load('data/train_data.npy')

In [ ]:
def train(D):

    # Hyper-parameter
    n_steps = ...
    learning_rate = ...
    input_size = 13  # Anzahl Merkmale (features)
    output_size = 1
    
    # Daten vorbereiten
    X = D[:, :-1].astype(np.float32)
    labels = D[:, -1].astype(np.float32)
    X = torch.from_numpy(X)
    y = torch.from_numpy(labels)

    # Modell definieren
    class LogisticRegression(nn.Module):
        def __init__(self, input_size):
            super().__init__()
            self.linear = nn.Linear(input_size, output_size)  # Xw (linear layer)

        def forward(self, x):
            out = self.linear(x)
            return out
    model = LogisticRegression(input_size)

    # loss
    # Dokumentation: https://pytorch.org/docs/stable/nn.html#torch.nn.BCEWithLogitsLoss
    criterion = nn.BCEWithLogitsLoss()  # Sigmoid und Binary-Cross-Entropy-Loss

    # optimizer
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Alternative zu Adam
    # Dokumentation: https://pytorch.org/docs/stable/optim.html#torch.optim.SGD
    # momentum = 0.9  # Wert zwischen 0. und 1.
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    # trainieren des Modells
    for e in range(n_steps):
        # forward pass
        outputs = model.forward(X)[:, 0]
        cost = criterion(outputs, y)

        # backward pass (berechnet die Gradienten automatisch)
        optimizer.zero_grad()  # reset gradients (torch akkumuliert Gradienten)
        cost.backward()  # berechnen der Gradienten

        # Optimierungsschritt durchfuehren
        optimizer.step()

        # Berechnung der Accuracy
        pred_labels = outputs > 0
        is_correct = torch.eq(pred_labels, y.byte()).float()
        accuracy = torch.mean(is_correct).item()

    return model
